In [ ]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import polars as pl
import pandas as pd
from sklearn.model_selection import GroupKFold,StratifiedGroupKFold
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
import re
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import joblib

import kaggle_evaluation.mcts_inference_server

import pickle

with open('/kaggle/input/mcts-data/winrate_dict_updated.pkl', 'rb') as f:##
    winrate_dict = pickle.load(f)
    
with open('/kaggle/input/mcts-data/win_ruleset_dict.pkl', 'rb') as f:
    win_rule_dict = pickle.load(f)
    
with open('/kaggle/input/mcts-data/good_features.pkl', 'rb') as f:
    good_features = pickle.load(f)

deberta_oof_df = pd.read_csv("/kaggle/input/deberta-mcts-models/valid_df_v1.csv")
ruleset_pred_dict = dict(zip(deberta_oof_df['GameRulesetName'], deberta_oof_df['pred']))

utility_diff_pairs = pd.read_csv("/kaggle/input/mcts-data/performance_diff_reduced.csv")
utility_diff_dict = dict(zip(utility_diff_pairs['sorted_pair'], utility_diff_pairs['performance_difference']))#

In [ ]:
columns_to_drop = ['HexShape','MorrisTiling','AlquerqueTiling','TriangleTiling','ConcentricTiling','PolygonShape',
                   'SquareTiling','SquareShape','RegularShape','TriangleShape','HexShape','SquarePyramidalShape']

constant_cols = pd.read_csv('/kaggle/input/um-gps-of-mcts-variants-constant-columns/constant_columns.csv').columns.to_list()
target_col = 'utility_agent1'

agent_cols = ['agent1', 'agent2','p1_selection','p1_exploration','p1_playout','p1_bounds','p2_selection'
              ,'p2_exploration','p2_playout','p2_bounds']
good_features_reduced = [string for string in good_features if string not in agent_cols]
good_features = [string for string in good_features if string not in columns_to_drop]

extra_cols = ['ARI','CLRI','McAlpine_EFLAW']
more_cols = ['oof_preds','deberta_pred','winrate','winrate_2']

columns_to_analyze = [
    'ScoreDifferenceAverage', 'ScoreDifferenceMedian', 'ScoreDifferenceMaximum',
    'ScoreDifferenceVariance', 'ScoreDifferenceChangeAverage', 'ScoreDifferenceChangeLineBestFit',
    'ScoreDifferenceMaxIncrease', 'ScoreDifferenceMaxDecrease'
]

In [ ]:
def ARI(txt):
    characters=len(txt)
    words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))
    sentence=len(re.split('\\.|\\?|\\!',txt))
    ari_score=4.71*(characters/words)+0.5*(words/sentence)-21.43
    return ari_score

def McAlpine_EFLAW(txt):
    W=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))
    S=len(re.split('\\.|\\?|\\!',txt))
    mcalpine_eflaw_score=(W+S*W)/S
    return mcalpine_eflaw_score

def CLRI(txt):
    characters=len(txt)
    words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))
    sentence=len(re.split('\\.|\\?|\\!',txt))
    L=100*characters/words
    S=100*sentence/words
    clri_score=0.0588*L-0.296*S-15.8
    return clri_score

def preprocess_data(df_polars):
    df_polars = df_polars.drop(constant_cols)
    df = df_polars.with_columns(
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 1).alias('p1_selection'),
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 2).alias('p1_exploration').cast(pl.Float32),
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 3).alias('p1_playout'),
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 4).alias('p1_bounds'),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 1).alias('p2_selection'),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 2).alias('p2_exploration').cast(pl.Float32),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 3).alias('p2_playout'),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 4).alias('p2_bounds')
    ).to_pandas()

    target = df['utility_agent1']

    groups = df['GameRulesetName']
    
    df['winrate'] = df['agent1'].map(winrate_dict)
    df['winrate_2'] = df['agent2'].map(winrate_dict)

    derberta_preds = df['GameRulesetName'].map(ruleset_pred_dict)
    
    df['ARI'] = df['EnglishRules'].apply(ARI)
    df['McAlpine_EFLAW'] = df['EnglishRules'].apply(McAlpine_EFLAW)
    df['CLRI'] = df['EnglishRules'].apply(CLRI)
    
    df['PlayoutsPerSecond']=df['PlayoutsPerSecond'].clip(0,25000)
    df['MovesPerSecond']=df['MovesPerSecond'].clip(0,1000000)
    
    df['pair'] = df['agent1'] + df['agent2']

    df[columns_to_analyze] = df[columns_to_analyze].apply(lambda x: np.log1p(x - x.min() + 1))
    df['agent1']= df.agent1.str.rsplit('-', n=1).str[0]
    df['agent2']= df.agent2.str.rsplit('-', n=1).str[0]
    df['sorted_pair'] = df.apply(lambda x: '_'.join(sorted([x['agent1'], x['agent2']])), axis=1)
    df['sorted_win_ratio'] = df['sorted_pair'].map(utility_diff_dict)

    cat_features = df.columns[df.dtypes == object]
    
    for feature in cat_features:
        dtype = pd.CategoricalDtype(categories=list(set(df[feature])))
        df[feature] = df[feature].astype(dtype)
     
    cat_features = cat_features.tolist()
    
    cat_features = [string for string in cat_features if string in good_features]
    
    return df, target, groups,derberta_preds,cat_features

def preprocess_test(df_polars):
    df_polars = df_polars.drop(constant_cols)
    
    df = df_polars.with_columns(
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 1).alias('p1_selection'),
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 2).alias('p1_exploration').cast(pl.Float32),
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 3).alias('p1_playout'),
        pl.col('agent1').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 4).alias('p1_bounds'),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 1).alias('p2_selection'),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 2).alias('p2_exploration').cast(pl.Float32),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 3).alias('p2_playout'),
        pl.col('agent2').str.extract(r'MCTS-(.*)-(.*)-(.*)-(.*)', 4).alias('p2_bounds')
    ).to_pandas()
    
    df['winrate'] = df['agent1'].map(winrate_dict)
    df['winrate_2'] = df['agent2'].map(winrate_dict)

    df['ARI'] = df['EnglishRules'].apply(ARI)
    df['McAlpine_EFLAW'] = df['EnglishRules'].apply(McAlpine_EFLAW)
    df['CLRI'] = df['EnglishRules'].apply(CLRI)
    
    df['PlayoutsPerSecond']=df['PlayoutsPerSecond'].clip(0,25000)
    df['MovesPerSecond']=df['MovesPerSecond'].clip(0,1000000)
    
    
    df['pair'] = df['agent1'] + df['agent2']

    df[columns_to_analyze] = df[columns_to_analyze].apply(lambda x: np.log1p(x - x.min() + 1))

    df.drop(columns=['GameRulesetName', 'EnglishRules', 'LudRules'], inplace=True)

    cat_features = df.columns[df.dtypes == object]
    for feature in cat_features:
        dtype = pd.CategoricalDtype(categories=list(set(df[feature])))
        df[feature] = df[feature].astype(dtype)

    return df

def infer_lgb(data, models):
    return np.mean([model.predict(data) for model in models], axis=0)

In [ ]:
#Isnt used here, its just the training function for the main model
def train(data, target, groups, cat_features):
    X = data[good_features + extra_cols + more_cols]
    y = target
    
    y_int = data['pair']
    
    cv = StratifiedGroupKFold(n_splits=Config.n_splits,random_state=2024,shuffle=True)
    
    models = []
    best_scores = []
    oof_preds = np.zeros(len(data))

    for fi, (train_idx, valid_idx) in enumerate(cv.split(X,y_int, groups)):
        print(f'Fold {fi+1}/{Config.n_splits} ...')
        model = CatBoostRegressor(**cb_params, 
                                  cat_features=data[good_features+ more_cols].columns[data[good_features+ more_cols].dtypes == 'category'].values)
        
        model.fit(X.iloc[train_idx], y.iloc[train_idx],
                  eval_set=[(X.iloc[valid_idx], y.iloc[valid_idx])],
                  early_stopping_rounds=100)
        
        models.append(model)
        
        best_validation_score = model.best_score_['validation']['RMSE']
        best_scores.append(best_validation_score)
        print(f"Best validation score for fold {fi+1}: {best_validation_score}")
        
        oof_preds[valid_idx] = model.predict(X.iloc[valid_idx])

    oof_df = pd.DataFrame({
        'true_values': y,
        'oof_predictions': oof_preds
    })

    avg_score = sum(best_scores) / len(best_scores)
    print(f'Average validation score across {Config.n_splits} folds: {avg_score}')
    
    return models,oof_df

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
VER=1
LOAD_FROM = '/kaggle/input/deberta-mcts-models/'

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import string

class Tokenize(object):
    def __init__(self, valid, tokenizer):
        self.tokenizer = tokenizer
        self.valid = valid
        
    def get_dataset(self, df):
        ds = Dataset.from_dict({
            'essay_id': df.index.to_list(),
            'full_text': df['LudRules'].tolist(),
        })
        return ds
        
    def tokenize_function(self, example):
        return self.tokenizer(
            example['full_text'], truncation=True, max_length=3072#CFG.max_length###
        )
    
    def __call__(self):
        valid_ds = self.get_dataset(self.valid)
        tokenized_valid = valid_ds.map(
            self.tokenize_function, batched=True)
        
        return tokenized_valid, self.tokenizer

training_args = TrainingArguments(".", per_device_eval_batch_size=1, report_to="none")

tokenizer = AutoTokenizer.from_pretrained(LOAD_FROM + f'deberta-v3-small_MCTS_fold_0_v{VER}')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(LOAD_FROM + f'deberta-v3-small_MCTS_fold_0_v{VER}')
model_2 = AutoModelForSequenceClassification.from_pretrained(LOAD_FROM + f'deberta-v3-small_MCTS_fold_1_v{VER}')
model_3 = AutoModelForSequenceClassification.from_pretrained(LOAD_FROM + f'deberta-v3-small_MCTS_fold_2_v{VER}')
model_4 = AutoModelForSequenceClassification.from_pretrained(LOAD_FROM + f'deberta-v3-small_MCTS_fold_3_v{VER}')

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def extract_end_section(text):
    end_idx = text.rfind("(end")
    
    if end_idx != -1:
        return text[end_idx:]
    else:
        return tex

In [ ]:
run_i = 0

def predict(test_data, submission):
    global run_i, models, pre_models, oof_df,tokenizer,model,TRAIN_MODELS,seeds,LOAD_PRETRAIN
    if run_i == 0:
        pre_models = joblib.load('/kaggle/input/mcts-pretrain-data/pre_models_array.pkl')
        oof_df = pd.read_csv('/kaggle/input/mcts-pretrain-data/oof_predictions.csv')
        models = joblib.load('/kaggle/input/mcts-data/models_array.pkl')
                     
    run_i += 1
    
    test_data = test_data.to_pandas()
    test_data['LudRules'] = test_data['LudRules'].apply(extract_end_section)
    test_data['LudRules'] = test_data['LudRules'].apply(remove_punctuation)
    tokenize = Tokenize(test_data, tokenizer)
    tokenized_test, _ = tokenize()
    def infer_with_trainer(model):
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_test,
            data_collator=data_collator,
            tokenizer=tokenizer,
        )
        return trainer.predict(tokenized_test).predictions
    
    predictions = infer_with_trainer(model)
    predictions_2 = infer_with_trainer(model_2)
    predictions_3 = infer_with_trainer(model_3)
    predictions_4 = infer_with_trainer(model_4)
    test_data = pl.from_pandas(test_data)
    
    test_data = preprocess_test(test_data)
    test_data['oof_preds'] = infer_lgb(test_data[good_features_reduced+extra_cols], pre_models)
    test_data['deberta_pred'] = predictions*0.25+predictions_2*0.25+predictions_3*0.25+predictions_4*0.25
    prediction_cat = infer_lgb(test_data[good_features + extra_cols + more_cols], models)
    clipped_predictions = np.clip(prediction_cat*1.15, -1.0, 1.0)

    # Return the submission with clipped predictions
    return submission.with_columns(pl.Series(target_col, clipped_predictions ))#discrete_predictions

inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )